# Distribution des naissances – As de la Guerre


Ce carnet propose une analyse de la distribution dans le temps des années de naissance de la population des **As de la Première Guerre mondiale**, en s'appuyant sur les données stockées dans un triplestore **AllegroGraph** personnel et hébergé dans le cloud.

L'objectif est de :
- Visualiser la distribution temporelle des naissances
- Mettre en évidence des générations
- Étudier l'évolution du genre dans le temps


In [ ]:

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

import sparql_functions as spqf


In [ ]:

endpoint = "https://ag18eeehwepcg5xz.allegrograph.cloud/repositories/As-de-la-Guerre"

query = '''
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?s (MAX(?label) as ?label) (xsd:integer(MAX(?birthDate)) as ?birthYear) (MAX(?gen) as ?genderUri)
WHERE {
  ?s wdt:P106 wd:Q2095549 .
  ?s wdt:P607 wd:Q361 .
  FILTER NOT EXISTS { ?s wdt:P31 wd:Q15632617. }
  ?s rdfs:label ?label .
  ?s wdt:P569 ?birthDate .
  OPTIONAL { ?s wdt:P21 ?gen . }
}
GROUP BY ?s
'''


In [ ]:

qr = spqf.get_json_sparql_result(endpoint, query)
r = [l for l in spqf.sparql_result_to_list(qr)]
print(f"{len(r)} résultats extraits.")
r[:5]


In [ ]:

df = pd.DataFrame(r)
df.columns = ['personUri', 'personLabel', 'birthYear', 'genderUri']
df['birthYear'] = df['birthYear'].astype(int)
df.head()


In [ ]:

def code_gender(g_uri):
    if g_uri == 'http://www.wikidata.org/entity/Q6581097':
        return 'M'
    elif g_uri == 'http://www.wikidata.org/entity/Q6581072':
        return 'F'
    else:
        return 'A'

df['genCode'] = df.apply(lambda x: code_gender(x['genderUri']), axis=1)


In [ ]:

years_count = pd.DataFrame(df[df['birthYear'] < 2001].groupby('birthYear').size()).reset_index()
years_count.columns = ['year', 'effectif']

ax = years_count.plot(x='year', y='effectif', kind='bar', rot=60, fontsize=8, figsize=(16, 6))
plt.ylabel('Effectif')
plt.xlabel('Années')
plt.title('Nombre de naissances par année')
plt.show()


In [ ]:

cdf = df[df['birthYear'] < 1951].copy()
l_25 = list(range(1751, 1975, 25))
cdf['periodes'] = pd.cut(cdf['birthYear'], l_25, right=False)
cdf['periodes'] = cdf['periodes'].apply(lambda x: str(int(x.left)) + '-' + str(int(x.right)-1))


In [ ]:

ax = cdf.groupby('periodes').size().plot(kind='bar', rot=60, figsize=(16, 6))
plt.ylabel('Effectif')
plt.xlabel('Périodes')
plt.title('Naissances par périodes de 25 ans')
plt.show()


In [ ]:

gender_m = pd.DataFrame(cdf[cdf.genCode == 'M'].groupby('periodes').size())
gender_m.columns = ['gen_m']

gender_f = pd.DataFrame(cdf[cdf.genCode == 'F'].groupby('periodes').size())
gender_f.columns = ['gen_f']

merged = pd.merge(gender_m, gender_f, left_index=True, right_index=True)
merged.index = merged.index.map(str)


In [ ]:

ax = merged.plot.bar(stacked=True, rot=70, fontsize=9, figsize=(20, 6), color=['LightSteelBlue', 'LightCoral'])
plt.title('Répartition des genres par période')
plt.ylabel('Effectif')
plt.xlabel('Périodes')
plt.show()


In [ ]:

os.makedirs('images', exist_ok=True)
plt.savefig('images/naissances_distribution_25ans_genre.jpg', bbox_inches='tight', pad_inches=0.2)
